In [1]:
import random
import time
from bs4 import BeautifulSoup 
import re
from MongoDb import MongoDb
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

db = MongoDb('mongodb://localhost:27017')

In [ ]:
# scraping names and urls of all cards
driver = webdriver.Chrome()

def check_exists_by_xpath(xpath):
    try:
        driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[2]/div/div[3]/div/div[3]/button')
    except NoSuchElementException:
        return False
    return True

driver.get('https://www.lotto.pl/zdrapki/katalog-zdrapek')
WebDriverWait(driver, 2000).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/div/div/div[2]/div/div[4]/div[2]/div[3]/div/button'))).click()
WebDriverWait(driver, 2000).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/div/div/div[2]/div/div[2]/div/div/div[2]/button'))).click()
time.sleep(3)

while(check_exists_by_xpath('/html/body/div[1]/div/div/div[2]/div/div[3]/div/div[3]/button')):
    WebDriverWait(driver, 2000).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/div/div/div[2]/div/div[3]/div/div[3]/button'))).click()
    time.sleep(1)
content = driver.page_source
driver.close()
soup = BeautifulSoup(content)

card_divs = soup.find_all('div', class_='scratch-box__item')
card_names_and_urls = []
for card in card_divs:
    card_url = card.findChild('a').get('href')
    card_name = card.findChild('div', class_='scratch-card-item__name').findChild('strong').get_text()
    card_names_and_urls.append({'name': card_name, 'url': card_url})
print(card_names_and_urls)

In [ ]:
#save result to avoid creating more requests
import json
with open('scratch_card.json', 'w') as outfile:
    json.dump(card_names_and_urls, outfile)

In [2]:
#load result
import json
with open('scratch_card.json') as json_file:
    card_names_and_urls = json.load(json_file)
    print(type(card_names_and_urls), card_names_and_urls)

<class 'list'> [{'name': 'OWOCOWE 8', 'url': '/zdrapki/katalog-zdrapek/owocowe-8,1325'}, {'name': 'MEGA PENSJA', 'url': '/zdrapki/katalog-zdrapek/mega-pensja,1324'}, {'name': 'MIĘTOWA KASA', 'url': '/zdrapki/katalog-zdrapek/mietowa-kasa,1323'}, {'name': 'WALIZKA PIENIĘDZY', 'url': '/zdrapki/katalog-zdrapek/walizka-pieniedzy,1322'}, {'name': 'JADĘ DO VEGAS', 'url': '/zdrapki/katalog-zdrapek/jade-do-vegas,1321'}, {'name': 'KRZYŻÓWKA XXL', 'url': '/zdrapki/katalog-zdrapek/krzyzowka-xxl,1320'}, {'name': 'SZCZĘŚLIWE LINIE', 'url': '/zdrapki/katalog-zdrapek/szczesliwe-linie,1319'}, {'name': 'SUPER MEGA PENSJA', 'url': '/zdrapki/katalog-zdrapek/super-mega-pensja,1318'}, {'name': 'LUCKY DAY', 'url': '/zdrapki/katalog-zdrapek/lucky-day,1317'}, {'name': 'PENSJA', 'url': '/zdrapki/katalog-zdrapek/pensja,1316'}, {'name': 'KIERUNEK KASA', 'url': '/zdrapki/katalog-zdrapek/kierunek-kasa,1315'}, {'name': 'SUPER BONUS', 'url': '/zdrapki/katalog-zdrapek/super-bonus,1313'}, {'name': 'KRZYŻÓWKA', 'url': '

In [3]:
def get_scratch_card_info(scratch_card_name, scratch_card_url):
    #read info about single scratch_card
    driver = webdriver.Chrome()
    driver.get('https://www.lotto.pl'+scratch_card_url)
    content = driver.page_source
    time.sleep((random.random()+2)/2)
    driver.close()
    soup = BeautifulSoup(content)

    # scraping information
    res = soup.find(string=re.compile('wygrane:')).get_text().replace('\xa0', '')
    pot, circulation = re.findall('(\d+)', res)

    scratch_price_div = soup.find('div', class_='scratch__price')
    scratch_price = scratch_price_div.select('strong')[0].get_text().replace('\xa0', '').replace('zł', '')

    date_strong = soup.find('strong', class_='scratch__regulations-disclaimer').get_text()
    start_date = re.search(r'(\d+\.\d+\.\d+)', date_strong) 
    start_date = start_date.group(1) + ' 0:00:00'
    start_date = datetime.strptime(start_date,'%d.%m.%Y %H:%M:%S')
    
    scratch_card = {'name': scratch_card_name,
                    'url': scratch_card_url,
                    'scratch_price': int(scratch_price),
                    'total_scratch_price': int(scratch_price)*int(circulation),
                    'start_date': start_date,
                    'pot': int(pot),
                    'circulation': int(circulation),
                    'win_values': [],
                    'n_wins': [],
                    'wins_left': [],
                    'last_update': datetime.now()

                    }

    table = soup.select('table td')

    scratch_card['win_values'] = [ int(table[i].get_text().replace('\xa0', '').replace('zł', '').strip()) for i in range(3, len(table),3)]
    scratch_card['n_wins'] = [int(table[i].get_text()) for i in range(4, len(table),3)]
    for i in range(5, len(table),3):
        text = table[i].get_text()
        if text == '-':
            text = 0
        else:
            text = int(text)
        scratch_card['wins_left'].append(text)

    return scratch_card

In [ ]:
# test
name = card_names_and_urls[50]['name']
url = card_names_and_urls[50]['url']
print(name, url)
result = get_scratch_card_info(name, url)
print(result)

In [4]:
#scrap data into mongo
i = 1
for card in card_names_and_urls[:30]:
    name = card['name']
    url = card['url']
    result = get_scratch_card_info(name, url)
    print(db.scratch_cards.insert_one(result), i)
    i += 1


<pymongo.results.InsertOneResult object at 0x000001BE4635BEE0> 1
<pymongo.results.InsertOneResult object at 0x000001BE4803CC10> 2
<pymongo.results.InsertOneResult object at 0x000001BE46374DC0> 3
<pymongo.results.InsertOneResult object at 0x000001BE4634FB50> 4
<pymongo.results.InsertOneResult object at 0x000001BE480F2CA0> 5
<pymongo.results.InsertOneResult object at 0x000001BE47D090D0> 6
<pymongo.results.InsertOneResult object at 0x000001BE46347190> 7
<pymongo.results.InsertOneResult object at 0x000001BE48582130> 8
<pymongo.results.InsertOneResult object at 0x000001BE47CB4550> 9
<pymongo.results.InsertOneResult object at 0x000001BE48050280> 10
<pymongo.results.InsertOneResult object at 0x000001BE485841C0> 11
<pymongo.results.InsertOneResult object at 0x000001BE482856A0> 12
<pymongo.results.InsertOneResult object at 0x000001BE48195EB0> 13
<pymongo.results.InsertOneResult object at 0x000001BE48285670> 14
<pymongo.results.InsertOneResult object at 0x000001BE4635BEE0> 15
<pymongo.results.In